In [25]:
import tensorflow as tf
import h5py
import numpy as np
import sys

In [98]:
model = tf.keras.models.load_model('no_compress.h5')

In [99]:
def uniform_quantize(tensor, num_bits):
    min_val = np.min(tensor)
    max_val = np.max(tensor)
    quantization_levels = 2 ** num_bits
    delta = (max_val - min_val) / quantization_levels
    return np.round((tensor - min_val) / delta) * delta + min_val

def fake_q(tensor):
  return np.int8(np.round(tensor*255))

def dont_q(tensor):
  return np.array(tensor)

In [100]:
model_f = []
model_n = []
model_i = []

for layer in model.trainable_weights:
  #l=fake_q(layer)
  l=dont_q(layer)
  zero_indices = np.uint8(np.transpose(np.where(l != 0)))
  l_p = l[l!=0] 

  model_f.append(l)
  model_n.append(layer.name)
  model_i.append(zero_indices)
    
  

In [101]:
for i in range(len(model_n)):
  print(model_n[i])
  print(model_f[i])
  print(model_i[i])
  break

conv1/kernel:0
[[[[-1.22195996e-01 -3.65397334e-01 -3.19065452e-01  3.18834335e-02
    -5.00621438e-01 -3.73942219e-02 -2.54532546e-01  4.90595587e-02
    -4.91164401e-02  3.06858439e-02 -4.92420405e-01 -1.38461143e-01
     4.86993603e-02 -2.96086878e-01  4.17319946e-02 -1.61894262e-01
     2.95839831e-03  2.15503797e-01 -3.03413600e-01 -1.21988803e-01
     1.31889388e-01  1.99267827e-03 -5.51096536e-02  3.92540991e-01
     5.61480105e-01 -5.08945823e-01 -1.19208731e-01  1.60011157e-01
     2.26257741e-01  6.79914746e-03  2.74347216e-01  5.28641224e-01]
   [ 8.77397507e-03 -3.64090987e-02 -5.09979427e-01 -1.61577404e-01
    -4.33045894e-01 -4.85445946e-01 -5.92394099e-02  3.76549900e-01
     2.10056037e-01  3.19314331e-01 -6.93876743e-02 -1.15803450e-01
     2.86363631e-01  2.66886860e-01 -7.60857090e-02  1.04939647e-01
     3.00567389e-01 -7.14545250e-01 -5.36540188e-02  1.27990190e-02
    -2.76520461e-01 -8.59291926e-02 -1.00078434e-01  3.09738189e-01
     4.80800331e-01 -3.92912805e

In [102]:
for layer in model.trainable_weights:
  print(layer)
  break

<tf.Variable 'conv1/kernel:0' shape=(3, 3, 3, 32) dtype=float32, numpy=
array([[[[-1.22195996e-01, -3.65397334e-01, -3.19065452e-01,
           3.18834335e-02, -5.00621438e-01, -3.73942219e-02,
          -2.54532546e-01,  4.90595587e-02, -4.91164401e-02,
           3.06858439e-02, -4.92420405e-01, -1.38461143e-01,
           4.86993603e-02, -2.96086878e-01,  4.17319946e-02,
          -1.61894262e-01,  2.95839831e-03,  2.15503797e-01,
          -3.03413600e-01, -1.21988803e-01,  1.31889388e-01,
           1.99267827e-03, -5.51096536e-02,  3.92540991e-01,
           5.61480105e-01, -5.08945823e-01, -1.19208731e-01,
           1.60011157e-01,  2.26257741e-01,  6.79914746e-03,
           2.74347216e-01,  5.28641224e-01],
         [ 8.77397507e-03, -3.64090987e-02, -5.09979427e-01,
          -1.61577404e-01, -4.33045894e-01, -4.85445946e-01,
          -5.92394099e-02,  3.76549900e-01,  2.10056037e-01,
           3.19314331e-01, -6.93876743e-02, -1.15803450e-01,
           2.86363631e-01,  2

In [103]:

#import numpy as np

# Create sample tensors
#tensor1 = np.array([[1, 2, 3], [4, 5, 6]])
#tensor2 = np.array([[7, 8, 9], [10, 11, 12]])

# Create an H5 file
file_path = 'data_no_compress.h5'
with h5py.File(file_path, 'w') as hf:
    # Create a group
    group = hf.create_group('tensors')
    for i in range(len(model_n)):
      # Create datasets within the group
      group.create_dataset(model_n[i], data=model_f[i])
      group.create_dataset(str(model_n[i]+"_idx"), data=model_i[i])
      #break
      #group.create_dataset('tensor2', data=tensor2)


print('H5 file created successfully.')


H5 file created successfully.


In [105]:
model__ = []
names__ = []
# open the file as 'f'
with h5py.File('data_no_compress.h5', 'r') as f:
    data = f['tensors']
    #print(data.members)
    members = list(data.keys())
    #print("Members in the h5 file:")
    for member in members:
        #print(member)
        d = data[member]
        for m in list(d.keys()):
          dd = d[m]
          m_ = dd[()]
          model__.append(m_)
          names__.append(member+"/"+m)
          #print(type(m[0]))

print(names__[0])

conv1/bias:0


In [106]:
model__ = []
names__ = []
index__ = []

# open the file as 'f'
with h5py.File('data_no_compress.h5', 'r') as f:
    data = f['tensors']
    #print(data.members)
    members = list(data.keys())
    #print("Members in the h5 file:")
    for member in members:
        #print(member)
        d = data[member]
        for m in list(d.keys()):
          dd = d[m]
          m_ = dd[()]
          if "idx" not in m:
          	model__.append(m_)
          	#names__.append(member+"/"+m)
          else:
          	index__.append(m_.astype(np.int8))
print(index__[1].dtype)
print(model__[1].dtype)

int8
float32


In [107]:
total_size = sys.getsizeof(model__) + sum(sys.getsizeof(arr) for arr in model__) + sum(sys.getsizeof(arr) for arr in index__)
print(total_size)

7050842
